# TTA Debug Notebook
Visualise the SAM2 TTA pipeline: soft teacher construction, region partition, student predictions, and losses (anchor / entropy / consistency) including augmented view.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
SAM2_ROOT = ROOT.parent / "sam2"
if SAM2_ROOT.exists() and str(SAM2_ROOT) not in sys.path:
    sys.path.insert(0, str(SAM2_ROOT))

from configs.pipeline_config import load_pipeline_config
from debug_tests.debug_test import MAIN_DIR, _load_constants
from image_processings.tta import TTAPipeline, TTALossWeights
from image_processings.tta.tta_core import default_multi_view_augment

%matplotlib inline

In [ ]:
# Replace these placeholders with actual outputs from the main pipeline
constants = _load_constants()
pipeline_cfg = load_pipeline_config(MAIN_DIR / constants["pipeline_cfg"])

pseudo_mask = np.zeros((64, 64), dtype=bool)  # pseudo label mask
image = np.zeros((64, 64, 3), dtype=np.uint8)  # input image
prompts = {
    "point_coords": np.array([[32, 32]], dtype=np.float32),
    "point_labels": np.array([1], dtype=np.int64),
    "box": None,
    "mask_input": None,
    "multimask_output": False,
}


# Dummy predictor with required API; replace with SAM2 predictor in practice
class DummyPredictor:
    def predict(self, point_coords, point_labels, box, mask_input, multimask_output, return_logits=False):
        logits = np.random.randn(1, teacher.shape[0], teacher.shape[1]).astype(np.float32)
        return logits, [1.0], None

predictor = DummyPredictor()
pipeline = TTAPipeline(predictor=predictor, loss_weights=TTALossWeights(), augment_fn=default_multi_view_augment())
tta_out = pipeline.step(image, prompts, pseudo_mask)


In [ ]:
# Visualise pseudo-mask and predictions
import matplotlib.pyplot as plt

def show_mask(ax, mask, title, cmap="magma"):
    ax.imshow(mask, cmap=cmap)
    ax.set_title(title)
    ax.axis("off")

fig, axes = plt.subplots(1, 3, figsize=(9, 3))
show_mask(axes[0], pseudo_mask, "Pseudo mask", cmap="gray")
show_mask(axes[1], tta_out.student_probs, f"Student probs
Loss total={tta_out.losses['total']:.4f}")
if tta_out.student_probs_aug is not None:
    show_mask(axes[2], tta_out.student_probs_aug, "Augmented probs")
plt.show()

print("Loss breakdown:", tta_out.losses)
